# Hands-on 5: Plugins

## Introduction to plugins
In order to simplify the design of applications, QLM comes with an additional API, called `Plugin`. `Plugins` are object that can process a flow of quantum circuits (or jobs) on their way to a `QPU`, and/or process a flow of information (samples or values) on their way back from a `QPU`.

Their API is composed of two methods:
- **compile** for the way in (i.e from the user to the `QPU`). This method will take a `Batch` together with some `HardwareSpecs` and return a new `Batch`.
- **post_process** for the way out (i.e from the `QPU` to the user). This method will process a `BatchResult` and return either a `BatchResult` or a new `Batch` that should go back to the `QPU`.

This simple semantics allow to compose `Plugin` to form expressive compilation stacks and extend the features provided by a `QPU`.

Creating a stack using plugins is quite straightforward:

> my_stack = plugin1 | plugin2 | ... | my_qpu

In this code, when a fresh batch or job is submitted, the batch will run through the `compile()` of *plugin1*, the resulting batch will run through the `compile()` of *plugin2*, etc. When reaching my_qpu the execution will start, and the results will be processed in reversed order on their way back.

Overall the information flow can be summed up as follows:

![figure](./plugins.png)

## Writing your own plugin
**GOAL:** In the following jupyter our goal is to implement a plugin that erases every pattern of two consecutive hadamard gates (when on the same qubit).

The first step, as usual, is to import some classes.

We first need `Batch` and `HardwareSpecs` objects from `qat.core`:
- A `Batch` object contains a list of jobs that allows to send several circuits to a QPU with only a single request to the QPU or to a stack.
- A `HardwareSpecs` object contains the information from the Hardware like the number of qubits or the topology.

Secondly, we need the function `get_syntax` from `qat.core.util`. `get_syntax` will be usefull to obtain the information of the circuit from a job.

Last, we need `AbstractPlugin` object from qat.core.plugins to inherit our plugin from it.

In [ ]:
from XXX.XXX import XXX, XXX
from XXX.XXX.XXX import XXX
from XXX.XXX.XXX import XXX

In the next cell we will create the plugin to suppress two consecutive hadamard gates (on the same qubit).

To do so:
- Find a name for your plugin.
- Complete the `compile` function so it modifies and returns the modified batch. 

`get_syntax(circuit, index)` allows to get information of the instruction at this `index`. The result is an array containing three components:
- Name of the gate applied
- List of the angles used
- List of the qubits on which its applied

Example: 

- An Hadamard gate (applied on the first qubit of the program) will look like that ('H', [], [0]).
- A control-NOT gate (with the first qubit of the program controlling the operation) will look like that ('CNOT', [], [0, 1])

In [ ]:
## Our Plugin should inherit from the AbstractPlugin class:
class XXX(AbstractPlugin):
    ## Plugins must implement a compile method
    ## here batch will be a Batch object (see doc) that contains a list of Jobs.
    def compile(self, batch : Batch, specs : HardwareSpecs) -> Batch:
        ##We need do apply our plugin to all the jobs in our batch.
        for count, job in enumerate(batch.jobs):
            ##last_op is a vector. Its size is equal to the number of qubits in the circuit.
            ##This vector is used to specify if the last operation was an Hadamard gate or not.
            ##If 0, it is not an Hadamard otherwise it should contain the index+1 of the operation's line.
            last_op = np.zeros(job.circuit.nbqbits, dtype=int)
            index = 0
            ##Loop over all the operations.
            while index < len(job.circuit.ops):
                ##Get the syntax of the operation at number "index"
                syntax = get_syntax(job.circuit, XXX)
                ##Test if the gate is a Hadamard gate.
                if syntax[XXX] == 'XXX':
                    ##If the last operation is a Hadamard gate.
                    if last_op[syntax[2][0]] != XXX:
                        ##Suppress the line of last op
                        job.circuit.ops.pop(last_op[syntax[XXX][XXX]]-1)
                        
                        ##Actualize the index
                        index = XXX - 1
                        
                        ##Suppress the line of index
                        job.circuit.ops.pop(XXX)
                        
                        ##Actualise indices of the vector last_op since an operation has been suppressed
                        ##Loop over all indices of the vector last_op
                        for j in range(len(XXX)):
                            if last_op[j] > last_op[syntax[2][0]]:
                                last_op[j] = last_op[j] - 1
                        last_op[syntax[2][0]] = 0
                        index = index - 1
                    ##If the last operation is a Hadamard gate
                    else:
                        ##Actualise last op with the right index
                        last_op[syntax[2][0]] = XXX + 1
                ##If the gate is not a Hadamard gate.
                else:
                    ##Actualize the last operation
                    for i in syntax[2]:
                        last_op[i] = XXX 
                index = index + 1
        return batch
    
    ## We do not want to do anything in post-processing.
    def do_post_processing(self) -> bool:
        return True
    
    def post_process(self, batch_result):
        return batch_result

To test our plugin we need to create a simple test case:

In [ ]:
from qat.lang.AQASM import *
prog = Program()
qbits = prog.qalloc(1)

prog.apply(H, qbits[0])
prog.apply(H, qbits[0])

circ = prog.to_circ()

%qatdisplay circ


Now we need to create the stack:
- Define the name of your stack.
- Use your plugin in the stack.
- We will use LinAlg as a simulator.

Once that's done we need to create and submit the job to our stack.

In [ ]:
from qat.qpus import LinAlg
import numpy as np
XXX = XXX() | LinAlg()

job = circ.to_job()
results = XXX.submit(job)

%qatdisplay circ

At the end you should be able to modify the circuit to suppress the right gates.
I Advise you to modify the test circuit to verify your plugin:
- Add qubits
- Add gates